# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd




In [2]:
pip install pandas requests lxml


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:

url = "https://en.wikipedia.org/wiki/List_of_most-streamed_songs_on_Spotify"
headers = {"User-Agent": "Mozilla/5.0"} # esto se hace para parecer un navegador real
response = requests.get(url, headers=headers) # contiene toda la respuesta del servidor a tu petición HTTP
print(response.status_code) #es una forma de verificar que la petición fue exitosa (200)
response.raise_for_status() # detiene el programa si hubo un error
html = response.text

200


In [31]:
# convertimos el html en una sopa
soup = BeautifulSoup(response.content) # lo convertimos a sopa para poder recorrerlo más fácilmente
table = soup.find("table", {"class": "wikitable"}) # buscamos la tabla por su clase CSS
# Convertimos la tabla a DataFrame
df = pd.read_html(str(table))[0]
print("Tabla cargada. Primeras 5 filas:")
print(df.head())


Tabla cargada. Primeras 5 filas:
  Rank                 Song                 Artist(s) Streams (billions)  \
0    1    "Blinding Lights"                The Weeknd              5.084   
1    2       "Shape of You"                Ed Sheeran              4.601   
2    3            "Starboy"  The Weeknd and Daft Punk              4.153   
3    4    "Sweater Weather"         The Neighbourhood              4.102   
4    5  "Someone You Loved"             Lewis Capaldi              4.094   

        Release date Ref.  
0   29 November 2019  [1]  
1     6 January 2017  [2]  
2  21 September 2016  [3]  
3    3 December 2012  [4]  
4    8 November 2018  [5]  


/tmp/ipykernel_824/2542077442.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
